In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd

app = Flask(__name__)

@app.route('/intraday_pnl', methods=['POST'])
def intraday_pnl():
    try:
        data = request.get_json()

        stock = data.get("stock")
        start_date = data.get("start_date")
        end_date = data.get("end_date")
        entry_time = data.get("entry_time")
        exit_time = data.get("exit_time")

        if not all([stock, start_date, end_date, entry_time, exit_time]):
            return jsonify({"error": "Missing required parameters"}), 400

        # ----------------------------
        # DOWNLOAD 1-MIN DATA
        # ----------------------------
        df = yf.download(
            stock,
            start=start_date,
            end=end_date,
            interval="1m"
        )

        if df.empty:
            return jsonify({"error": "No data found for given inputs"}), 404

        df.reset_index(inplace=True)
        df["Date"] = df["Datetime"].dt.date
        df["Time"] = df["Datetime"].dt.strftime("%H:%M")

        daily_pnl = []
        unique_days = df["Date"].unique()

        for day in unique_days:
            day_df = df[df["Date"] == day]

            # Nearest entry time ≥ entry_time
            entry_row = day_df[day_df["Time"] >= entry_time].head(1)

            # Nearest exit time ≤ exit_time
            exit_row = day_df[day_df["Time"] <= exit_time].tail(1)

            if entry_row.empty or exit_row.empty:
                continue

            entry_price = float(entry_row["Open"].iloc[0])
            exit_price = float(exit_row["Close"].iloc[0])
            pnl = exit_price - entry_price

            daily_pnl.append({
                "Date": str(day),
                "Entry Price": entry_price,
                "Exit Price": exit_price,
                "PnL": pnl
            })

        total_pnl = sum([x["PnL"] for x in daily_pnl])

        return jsonify({
            "Daily_PnL": daily_pnl,
            "Total_PnL": total_pnl
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Nov/2025 11:58:17] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Nov/2025 11:58:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Nov/2025 11:58:27] "GET /intraday_pnl HTTP/1.1" 405 -
127.0.0.1 - - [24/Nov/2025 11:59:22] "POST /intraday_pnl HTTP/1.1" 500 -
C:\Users\user\AppData\Local\Temp\ipykernel_14740\3745201271.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed
127.0.0.1 - - [24/Nov/2025 12:00:15] "POST /intraday_pnl HTTP/1.1" 200 -
